# SCI6485 Introduction to Generative Artificial Intelligence

##  STYLEGAN3


**Harvard University**<br/>
**Fall 2023**<br/>
**Instructor:**  Sabrina Osmany<br/>
**TA:**  Jiabin Wei<br/>

**DISCLAIMER**: No public reproduction of this code is allowed without the explicit consent of their authors.

<hr style="height:2pt">

---

 This tutorial is heavily borrowed from this github [repo](https://github.com/derekphilipau/machinelearningforartists).

# Training custom data with StyleGAN

In this tutorial, we only want to familiarize ourselves with running StyleGAN (in this case, the newest version) and seeing how training progresses.

## Verify Runtime is GPU

In the menu, select Runtime -> Change Runtime Type and verify you are using the **GPU**.  Also select **High-RAM** if you are using Colab Pro.

The `nvidia-smi` command below should **NOT** display *"NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver."*

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-53bfe10e-fd15-86ab-8244-52a30d394d29)


## Mount your Google Drive

You will be storing the training models and progress images on your Google Drive.  This is very convenient for viewing progress, and if your Colab notebook is disconnected you will not lose your models.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

## Install Stylegan pytorch Prerequisites

In [ ]:
import torch
import torchvision
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Selected device: {device}')
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

## Get the StyleGAN code

In [ ]:
# clone StyleGan repo from github
!git clone https://github.com/NVlabs/stylegan3

Cloning into 'stylegan3'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 212 (delta 0), reused 1 (delta 0), pack-reused 207
Receiving objects: 100% (212/212), 4.16 MiB | 7.70 MiB/s, done.
Resolving deltas: 100% (101/101), done.


 create a dataset folder under your stylegan directory

In [ ]:
mkdir /content/stylegan3/datasets

## Load a custom dataset

In [ ]:
#######################################################
#      unzip the custome dataset zip file to the 'datasets' folder under the 'stylegan3' directory
#      you need to replace the path after 'unzip' with your zip file directory
#######################################################

# Custom datasets can be created from a folder containing images
# unzip your custom data zip file to stylegan3 dataset directory
!unzip /content/drive/MyDrive/MDE/SCI6485/StyleGan/custom_data/custom_data_2.zip -d /content/stylegan3/datasets

Archive:  /content/drive/MyDrive/MDE/SCI6485/StyleGan/custom_data/custom_data_2.zip
   creating: /content/stylegan3/datasets/custom_data 2/
  inflating: /content/stylegan3/datasets/__MACOSX/._custom_data 2  
  inflating: /content/stylegan3/datasets/custom_data 2/yellow-24.png  
  inflating: /content/stylegan3/datasets/__MACOSX/custom_data 2/._yellow-24.png  
  inflating: /content/stylegan3/datasets/custom_data 2/yellow-30.png  
  inflating: /content/stylegan3/datasets/__MACOSX/custom_data 2/._yellow-30.png  
  inflating: /content/stylegan3/datasets/custom_data 2/yellow-18.png  
  inflating: /content/stylegan3/datasets/__MACOSX/custom_data 2/._yellow-18.png  
  inflating: /content/stylegan3/datasets/custom_data 2/yellow-1.png  
  inflating: /content/stylegan3/datasets/__MACOSX/custom_data 2/._yellow-1.png  
  inflating: /content/stylegan3/datasets/custom_data 2/yellow-142.png  
  inflating: /content/stylegan3/datasets/__MACOSX/custom_data 2/._yellow-142.png  
  inflating: /content/style

In [ ]:
# remove the "__MACOSX" directory from a Unix-based file system, typically found in ZIP or compressed archives created on macOS.
!rm -rf /content/stylegan3/datasets/__MACOSX

In [ ]:
cd /content/stylegan3

/content/stylegan3


In [ ]:
ls

avg_spectra.py   dnnlib/          gen_images.py  LICENSE.txt   training/
calc_metrics.py  Dockerfile       gen_video.py   metrics/      train.py
datasets/        docs/            gui_utils/     README.md     visualizer.py
dataset_tool.py  environment.yml  legacy.py      torch_utils/  viz/


In [ ]:
!pwd

/content/stylegan3


## Prepare your custom dataset for use by StyleGAN

In [ ]:
#######################################################
#      set --source= as the directory your dataset folder
#      set --dest= as the directory and the folder name of the new compressed zip file
#######################################################

# Datasets are stored as uncompressed ZIP archives containing uncompressed PNG files and a metadata file dataset.json for labels.
# !python dataset_tool.py --source=./datasets/custom_data --dest=./datasets/customdataset.zip

# you can also scale down the resolution of images here. You could use this line to check the explanation for the parameters: !python dataset_tool.py --help
!python dataset_tool.py --source=./datasets/custom_data_2 --dest=./datasets/customdataset.zip --resolution=32x32

Error: Missing input file or directory: ./datasets/custom_data_2


## Create Folders on Your Google Drive to Store Results

If we accidentally close our browser or the Colab runtime disconnects, we will lose all of our training models and progress images.  Therefore we want to store the training data on our Google Drive.  The following cells will help you create a new folder on your Google Drive and then your training data will be stored in this path later

In [ ]:
!mkdir /content/drive/MyDrive/MDE/SCI6485/StyleGan/custom_data_result_test

If you get an error "cannot create directory", it's probably because the folder already exists on your Google Drive and you can ignore the error.

## Train from Scratch

This cell block will train StyleGAN from scratch.  Training from scratch is much slower than using *transfer learning* on a previously trained model.  However, the purpose of this tutorial is to a) try training for the first time and b) notice how the progress images develop over time.

In [ ]:
# if you want check the meaning of the parameters in the train.py, you could run this code
!python train.py --help

In [ ]:
#######################################################
#      set --outdir= as the directory you want to store your training result
#      set --data= as the compressed zip file generated by the dataset_tool.py from above
#######################################################

# train the model
!python train.py --outdir=/content/drive/MyDrive/MDE/SCI6485/StyleGan/custom_data_result_test --cfg=stylegan3-t --data=./datasets/customdataset.zip  --gpus=1 --batch=32 --batch-gpu=4 --gamma=10.0 --mirror=1 --kimg=5000 --snap=1

Error: --data: [Errno 2] No such file or directory: './datasets/customdataset.zip'


In [ ]:
#######################################################
#      optional: fine tune a pretrained model
#                but you need to resize the images to fit the pre-trained model
#######################################################

# you could also fine a StyleGan3: Fine-tune StyleGAN3-R for MetFaces-U using 1 GPU, starting from the pre-trained FFHQ-U pickle.
!python train.py --outdir=/content/drive/MyDrive/MDE/SCI6485/StyleGan/custom_data_result --cfg=stylegan3-r --data=./datasets/customdataset.zip  --gpus=1 --batch=32 --batch-gpu=4 --gamma=10.0 --mirror=1 --kimg=5000 --snap=1 \
--resume=https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhqu-1024x1024.pkl

## Wait an Hour or Two

Normal training of a dataset can take days.  For this demonstration, we only want to train for a few hours in order to get familiar with the training process and see the initial results.

Results are stored in your Google Drive under your custom path defined above.

Each training run is stored in a separate directory.  The initial training run is stored in `00000-customdataset...`.  If you run training twice, the second run will be stored in `00001-customdataset...`.  And so on.

Inside the training run directory you will see various files.  `reals.png` shows a sample of the training dataset.  You should see various your custom images in this image. `fakes000000.png` is a sample of generated images from the initial model.  `network-snapshot-XXXXXX.pkl` is the actual model which can be used later to generate "fake" images, videos, etc.

As training progresses you will see more `fakes` & `network-snapshot` files created.

Since we only want to test a training, once you get to around the 500th iteration, you can stop the above training cell.  Then take a look at the `fakes` images and familiarize yourself with how the training progresses.



## Generate images

In [ ]:
#######################################################
#      set the directory where you want to store the new generated images
#######################################################

# make a directory to store newly generated images
!mkdir /content/drive/MyDrive/MDE/SCI6485/StyleGan/generated_images_2

mkdir: cannot create directory ‘/content/drive/MyDrive/MDE/SCI6485/StyleGan/generated_images’: File exists


In [ ]:
#######################################################
#      set --outdir= as the path above
#      set --network= as the training result snapshot you want to use for generating the image
#######################################################

# load the training result and generate a new image
# the new images will be stored in the path you created above
!python gen_images.py --outdir=/content/drive/MyDrive/MDE/SCI6485/StyleGan/generated_images_2 --trunc=1 --seeds=2 \
    --network=/content/drive/MyDrive/MDE/SCI6485/StyleGan/custom_data_2/00000-stylegan3-t-customdataset-gpus1-batch32-gamma10/network-snapshot-000012.pkl

Loading networks from "/content/drive/MyDrive/MDE/SCI6485/StyleGan/custom_data/00000-stylegan3-t-pokemondataset-gpus1-batch32-gamma10/network-snapshot-000012.pkl"...
Generating image for seed 2 (0/1) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.
